In [1]:
%pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [34]:
import requests
import pandas as pd
# import re
# import json
import ast

session = requests.Session()

In [4]:
# To scrape top 100 classic eyewear bestsellers data from Lenskart website (tot_num_of_products = 1006)
url = 'https://api-gateway.juno.lenskart.com/v2/products/category/8416?page-size=100&page=0'
res = session.get(url)
data = res.json()
data = data["result"]["product_list"]
df = pd.DataFrame(data)
cleaned_df = df.dropna(axis=1, how='all').drop(['imageUrls', 'suited_for','color_options','totalColorAvailable','specifications','arModelMsiteAndDesktopUrl','inclusiveText','relatedItems','prescriptionType'], axis=1).drop_duplicates(subset=['id'], keep='first').fillna('NULL').replace(r'^\s*$', 'NULL', regex=True)
cleaned_df.to_csv('classic_eyewear_bestsellers.csv', index=False)

In [35]:
# To scrape top 100 premium eyewear bestsellers data from Lenskart website (tot_num_of_products = 684)
url = 'https://api-gateway.juno.lenskart.com/v2/products/category/8080?page-size=100&page=0'
res = session.get(url)
data = res.json()
data = data["result"]["product_list"]
df = pd.DataFrame(data)
cleaned_df = df.dropna(axis=1, how='all').drop(['imageUrls', 'suited_for','color_options','totalColorAvailable','specifications','arModelMsiteAndDesktopUrl','inclusiveText','relatedItems','prescriptionType'], axis=1).drop_duplicates(subset=['id'], keep='first').fillna('NULL').replace(r'^\s*$', 'NULL', regex=True)
cleaned_df.to_csv('premium_eyewear_bestsellers.csv', index=False)

In [36]:
# To scrape all kids eyewear bestsellers data from Lenskart website (tot_num_of_products = 26)
url = 'https://api-gateway.juno.lenskart.com/v2/products/category/11261?page-size=100&page=0'
res = session.get(url)
data = res.json()
data = data["result"]["product_list"]
df = pd.DataFrame(data)
cleaned_df = df.dropna(axis=1, how='all').drop(['imageUrls', 'color_options','totalColorAvailable','specifications','arModelMsiteAndDesktopUrl','brand_logo','relatedItems','prescriptionType'], axis=1).drop_duplicates(subset=['id'], keep='first').fillna('NULL').replace(r'^\s*$', 'NULL', regex=True)
cleaned_df.to_csv('kids_eyewear_bestsellers.csv', index=False)

In [8]:
# To scrape trending classic eyewear  data from Lenskart website (tot_num_of_products = 4)
url = 'https://api-gateway.juno.lenskart.com/v2/products/category/7829?page-size=100&page=0'
res = session.get(url)
data = res.json()
data = data["result"]["product_list"]
df = pd.DataFrame(data)
cleaned_df = df.dropna(axis=1, how='all').drop(['imageUrls','color_options','totalColorAvailable','specifications','arModelMsiteAndDesktopUrl','relatedItems','prescriptionType'], axis=1).drop_duplicates(subset=['id'], keep='first').fillna('NULL').replace(r'^\s*$', 'NULL', regex=True)
cleaned_df.to_csv('classic_eyewear_trending.csv', index=False)

In [38]:
# To scrape top 100 classic sunglasses bestsellers data from Lenskart website (tot_num_of_products = 647)
url = 'https://api-gateway.juno.lenskart.com/v2/products/category/3909?page-size=100&page=0'
res = session.get(url)
data = res.json()
data = data["result"]["product_list"]
df = pd.DataFrame(data)
cleaned_df = df.dropna(axis=1, how='all').drop(['imageUrls', 'suited_for','color_options','totalColorAvailable','specifications','arModelMsiteAndDesktopUrl','inclusiveText','relatedItems','prescriptionType','brand_logo'], axis=1).drop_duplicates(subset=['id'], keep='first').fillna('NULL').replace(r'^\s*$', 'NULL', regex=True)
cleaned_df.to_csv('classic_sunglasses_bestsellers.csv', index=False)

In [39]:
# To scrape top 100 premium sunglasses bestsellers data from Lenskart website (tot_num_of_products = 249)
url = 'https://api-gateway.juno.lenskart.com/v2/products/category/19644?page-size=100&page=0'
res = session.get(url)
data = res.json()
data = data["result"]["product_list"]
df = pd.DataFrame(data)
cleaned_df = df.dropna(axis=1, how='all').drop(['imageUrls', 'suited_for','color_options','totalColorAvailable','specifications','arModelMsiteAndDesktopUrl','inclusiveText','relatedItems','prescriptionType','brand_logo'], axis=1).drop_duplicates(subset=['id'], keep='first').fillna('NULL').replace(r'^\s*$', 'NULL', regex=True)
cleaned_df.to_csv('premium_sunglasses_bestsellers.csv', index=False)

In [35]:
#To scrape the product data from product page
def get_product_details(url:str):
    session = requests.Session()
    try:
        res = session.get(url)
        source_page_content = res.text
        start_pattern = '"productDetailData":'
        end_pattern = '"cmsData":{'
        start_index = source_page_content.find(start_pattern)
        end_index = source_page_content.find(end_pattern,  start_index)
        if start_index != -1 and end_index != -1:
            json_data_str = source_page_content[start_index + len(start_pattern):end_index].rstrip(',')
            product_detail_data = json.loads(json_data_str)
            return product_detail_data
    except Exception as e:
        print("Error getting product data", e)
        return None

#To replace prices with Lenskart Price
def get_lenskart_price(prices_str):
    try:
        prices = ast.literal_eval(prices_str)
        for price_info in prices:
            if price_info['name'] == 'Lenskart Price':
                return price_info['price']
    except (ValueError, SyntaxError):
        return None

In [41]:
# To scrape the product data from product page

# Load the CSV data into a DataFrame
df = pd.read_csv('Contact_Lenses_Cleaned.csv')

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Get the product URL
    url = row['product_url']
    assert isinstance(url, str), f"Expected a string but got {type(url)}"

    # Get the product details as a JSON dictionary
    product_details = get_product_details(url)

    # If product_details is not None, add the new data to the DataFrame
    if product_details is not None:
        for key, value in product_details.items():
            # Format the column name
            column_name = key.lower().replace(' ', '_')
            if isinstance(value, (list, dict, pd.Series, pd.DataFrame)):
                value = str(value)
            # Add the new data to the DataFrame
            if column_name not in df.columns:
                df[column_name] = None
            df.loc[index, column_name] = value

# Save the updated DataFrame to a new CSV file
columns_to_drop = ['offerdetails', 'hashtagList', 'framedetails', 'additionaloptions', 'prescriptiontype', 'coloroptions', 'seo', 'generalproductinfo', 'technicalproductinfo', 'reviews', 'specifications', 'offerdetails' 'is_tbyb', 'image_url_suffix', 'brand_name_en', 'brandname', 'productmodelname', 'brand_logo', 'is_ditto', 'cmsLinkAndroid', 'cmsLinkIos', 'arModelAndroidUrl', 'arModelIOSUrl', 'arModelOriginalUrl', 'frameColorImage', 'isCygnusEnabled', 'sku', 'isquickcheckout', 'cmslinkandroid', 'relateditems',
                   'showproductrating', 'showpurchasecount', 'showwishlistcount', 'iscoloroptionextra', 'thumbnail', 'frontimage', 'coloroptionextracount', 'isditto', 'producturl', 'producturlwithoutdomain', 'gridimages', 'breadcrumb', 'cllegalreqdetails', 'jit', 'isplano', 'richcontent', 'istryonenabled', 'addtocartbuttons', 'imageurldetail', 'mobileimageurl', 'price', 'purchasecount', 'wishlistcount', 'productrating', 'totalnoofratings', 'subcategories', 'searchProductName', 'fullname', 'isCashbackApplicable', 'crosssells', 'powertypes']
# Only keep columns in the list that are in the DataFrame
columns_to_drop = [col for col in columns_to_drop if col in df.columns]

cleaned_df = df.dropna(axis=1, how='all').drop(columns_to_drop, axis=1).drop_duplicates(
    subset=['id'], keep='first').fillna('NULL').replace(r'^\s*$', 'NULL', regex=True)
cleaned_df['prices'] = cleaned_df['prices'].apply(get_lenskart_price)

cleaned_df.to_csv('Contact_Lenses_Updated.csv', index=False)

Error getting product data ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error getting product data HTTPSConnectionPool(host='www.lenskart.com', port=443): Max retries exceeded with url: /john-jacobs-jj-e14651-c3-eyeglasses.html (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000022CB6B0DA30>: Failed to resolve 'www.lenskart.com' ([Errno 11001] getaddrinfo failed)"))
Error getting product data HTTPSConnectionPool(host='www.lenskart.com', port=443): Max retries exceeded with url: /air-optix-colors-contact-lens-blue-2-lens-box.html (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000022CB6B0E8D0>: Failed to resolve 'www.lenskart.com' ([Errno 11001] getaddrinfo failed)"))
Error getting product data HTTPSConnectionPool(host='www.lenskart.com', port=443): Max retries exceeded with url: /air-optix-colors-contact-lens-pure-hazel-2-lens-box.html (Caused by NameResolutionError("<urlli